# Credit Risk Resampling Techniques

In [60]:
import warnings
warnings.filterwarnings('ignore')

In [156]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import balanced_accuracy_score
from imblearn.metrics import classification_report_imbalanced
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTEENN
from imblearn.under_sampling import ClusterCentroids

# Read the CSV into DataFrame

In [62]:
# Load the data
file_path = Path('Resources/lending_data.csv')
df = pd.read_csv(file_path)
df.head()

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,own,52800,0.431818,5,1,22800,low_risk
1,8400.0,6.692,own,43600,0.311927,3,0,13600,low_risk
2,9000.0,6.963,rent,46100,0.349241,3,0,16100,low_risk
3,10700.0,7.664,own,52700,0.430740,5,1,22700,low_risk
4,10800.0,7.698,mortgage,53000,0.433962,5,1,23000,low_risk


In [63]:
lending_binary_encoded = pd.get_dummies(df, columns=["homeowner"])
lending_binary_encoded.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status,homeowner_mortgage,homeowner_own,homeowner_rent
0,10700.0,7.672,52800,0.431818,5,1,22800,low_risk,0,1,0
1,8400.0,6.692,43600,0.311927,3,0,13600,low_risk,0,1,0
2,9000.0,6.963,46100,0.349241,3,0,16100,low_risk,0,0,1
3,10700.0,7.664,52700,0.430740,5,1,22700,low_risk,0,1,0
4,10800.0,7.698,53000,0.433962,5,1,23000,low_risk,1,0,0


# Split the Data into Training and Testing

In [64]:
# Create our features
X = lending_binary_encoded.copy()
X.drop('loan_status', axis=1, inplace=True)
#lending_binary_encoded = pd.get_dummies(loans_df, columns=["gender"])
#df.drop(columns=['loan_status','homeowner'])

# Create our target
le = LabelEncoder()
le.fit(lending_binary_encoded['loan_status'])
lending_binary_encoded["loan_status"] = le.transform(lending_binary_encoded['loan_status'])
y = lending_binary_encoded.loan_status.to_frame()

X.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,homeowner_mortgage,homeowner_own,homeowner_rent
0,10700.0,7.672,52800,0.431818,5,1,22800,0,1,0
1,8400.0,6.692,43600,0.311927,3,0,13600,0,1,0
2,9000.0,6.963,46100,0.349241,3,0,16100,0,0,1
3,10700.0,7.664,52700,0.430740,5,1,22700,0,1,0
4,10800.0,7.698,53000,0.433962,5,1,23000,1,0,0


In [65]:
y

,loan_status
0,1
1,1
2,1
3,1
4,1
...,...
77531,0
77532,0
77533,0
77534,0


In [66]:
X.describe()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,homeowner_mortgage,homeowner_own,homeowner_rent
count,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000
mean,9805.562577,7.292333,49221.949804,0.377318,3.826610,0.392308,19221.949804,0.497472,0.398911,0.103616
std,2093.223153,0.889495,8371.635077,0.081519,1.904426,0.582086,8371.635077,0.499997,0.489678,0.304764
min,5000.000000,5.250000,30000.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,8700.000000,6.825000,44800.000000,0.330357,3.000000,0.000000,14800.000000,0.000000,0.000000,0.000000
50%,9500.000000,7.172000,48100.000000,0.376299,4.000000,0.000000,18100.000000,0.000000,0.000000,0.000000
75%,10400.000000,7.528000,51400.000000,0.416342,4.000000,1.000000,21400.000000,1.000000,1.000000,0.000000
max,23800.000000,13.235000,105200.000000,0.714829,16.000000,3.000000,75200.000000,1.000000,1.000000,1.000000


In [68]:
# Check the balance of our target values
y['loan_status'].value_counts()

1    75036
0     2500
Name: loan_status, dtype: int64

In [93]:
# Create X_train, X_test, y_train, y_test
y = y.values
print(y)
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.33,random_state=1)
X_train.shape

[[1]
 [1]
 [1]
 ...
 [0]
 [0]
 [0]]


(51949, 10)

In [95]:
X_train

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,homeowner_mortgage,homeowner_own,homeowner_rent
55724,10900.0,7.759,53600,0.440299,5,1,23600,0,0,1
60387,10700.0,7.661,52700,0.430740,5,1,22700,0,0,1
23953,11300.0,7.943,55300,0.457505,5,1,25300,1,0,0
12782,8900.0,6.921,45700,0.343545,3,0,15700,1,0,0
46243,9500.0,7.152,47900,0.373695,4,0,17900,1,0,0
...,...,...,...,...,...,...,...,...,...,...
20609,7200.0,6.177,38700,0.224806,1,0,8700,0,1,0
21440,10000.0,7.389,50100,0.401198,4,1,20100,0,1,0
73349,10200.0,7.463,50800,0.409449,4,1,20800,0,1,0
50057,11100.0,7.838,54400,0.448529,5,1,24400,1,0,0


## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [96]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
data_scaler = StandardScaler()

In [97]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = data_scaler.fit(X_train)

In [98]:
# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [99]:
X_train

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,homeowner_mortgage,homeowner_own,homeowner_rent
55724,10900.0,7.759,53600,0.440299,5,1,23600,0,0,1
60387,10700.0,7.661,52700,0.430740,5,1,22700,0,0,1
23953,11300.0,7.943,55300,0.457505,5,1,25300,1,0,0
12782,8900.0,6.921,45700,0.343545,3,0,15700,1,0,0
46243,9500.0,7.152,47900,0.373695,4,0,17900,1,0,0
...,...,...,...,...,...,...,...,...,...,...
20609,7200.0,6.177,38700,0.224806,1,0,8700,0,1,0
21440,10000.0,7.389,50100,0.401198,4,1,20100,0,1,0
73349,10200.0,7.463,50800,0.409449,4,1,20800,0,1,0
50057,11100.0,7.838,54400,0.448529,5,1,24400,1,0,0


# Simple Logistic Regression

In [103]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_train_scaled, y_train)

LogisticRegression(random_state=1)

In [104]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = model.predict(X_test_scaled)
balanced_accuracy_score(y_test, y_pred)

0.9895615084227616

In [105]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0","Predicted 1"])
display(cm_df)

,Predicted 0,Predicted 1
Actual 0,830,13
Actual 1,135,24609


In [106]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.86      0.98      0.99      0.92      0.99      0.98       843
          1       1.00      0.99      0.98      1.00      0.99      0.98     24744

avg / total       0.99      0.99      0.98      0.99      0.99      0.98     25587



# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [123]:
# Resample the training data with the RandomOversampler
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train_scaled, y_train)

# View the count of target classes with Counter
Counter(y_resampled)

Counter({1: 50292, 0: 50292})

In [124]:
# Train the Logistic Regression model using the resampled data
lr_model = LogisticRegression(solver='lbfgs', random_state=1)
lr_model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [133]:
y_test
y_res_pred = lr_model.predict(X_test_scaled)

In [134]:
# Calculated the balanced accuracy score
balanced_accuracy_score(y_test, y_res_pred)

0.9943742068245021

In [135]:
# Display the confusion matrix
cm2 = confusion_matrix(y_test, y_res_pred)
cm2_df = pd.DataFrame(cm2, index=["Actual 0", "Actual 1"], columns=["Predicted 0","Predicted 1"])
display(cm2_df)

,Predicted 0,Predicted 1
Actual 0,839,4
Actual 1,161,24583


In [145]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_res_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.84      1.00      0.99      0.91      0.99      0.99       843
          1       1.00      0.99      1.00      1.00      0.99      0.99     24744

avg / total       0.99      0.99      1.00      0.99      0.99      0.99     25587



### SMOTE Oversampling

In [138]:
# Resample the training data with SMOTE
X_smote, y_smote = SMOTE(random_state=1, sampling_strategy=1.0).fit_resample(X_train_scaled, y_train)

# View the count of target classes with Counter
Counter(y_smote)

Counter({1: 50292, 0: 50292})

In [140]:
# Train the Logistic Regression model using the resampled data
lr_smote_model = LogisticRegression(solver='lbfgs', random_state=1)
lr_smote_model.fit(X_smote, y_smote)

LogisticRegression(random_state=1)

In [142]:
# Calculated the balanced accuracy score
y_smote_pred = lr_smote_model.predict(X_test_scaled)
balanced_accuracy_score(y_test, y_smote_pred)

0.9943944137433511

In [152]:
# Display the confusion matrix
cm_smote = confusion_matrix(y_test, y_smote_pred)
cm_smote_df = pd.DataFrame(cm_smote, index=["Actual 0", "Actual 1"], columns=["Predicted 0","Predicted 1"])
display(cm_smote_df)

,Predicted 0,Predicted 1
Actual 0,839,4
Actual 1,160,24584


In [154]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_smote_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.84      1.00      0.99      0.91      0.99      0.99       843
          1       1.00      0.99      1.00      1.00      0.99      0.99     24744

avg / total       0.99      0.99      1.00      0.99      0.99      0.99     25587



# Undersampling

In this section, you will test an undersampling algorithm to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [148]:
# Resample the data using the ClusterCentroids resampler
cc = ClusterCentroids(random_state=1)
X_undersampled, y_undersampled = cc.fit_resample(X_train_scaled, y_train)

# View the count of target classes with Counter
Counter(y_undersampled)

Counter({0: 1657, 1: 1657})

In [158]:
# Train the Logistic Regression model using the resampled data
undersample_lr_model = LogisticRegression(solver='lbfgs', random_state=1)
undersample_lr_model.fit(X_undersampled, y_undersampled)

LogisticRegression(random_state=1)

In [160]:
# Calculate the balanced accuracy score
y_under_pred = undersample_lr_model.predict(X_test_scaled)
balanced_accuracy_score(y_test, y_under_pred)

0.9937133470941732

In [162]:
# Display the confusion matrix
cm_undersample = confusion_matrix(y_test, y_under_pred)
cm_undersample_df = pd.DataFrame(cm_undersample, index=["Actual 0", "Actual 1"], columns=["Predicted 0","Predicted 1"])
display(cm_undersample_df)

,Predicted 0,Predicted 1
Actual 0,837,6
Actual 1,135,24609


In [163]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_under_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.86      0.99      0.99      0.92      0.99      0.99       843
          1       1.00      0.99      0.99      1.00      0.99      0.99     24744

avg / total       1.00      0.99      0.99      0.99      0.99      0.99     25587



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [157]:
# Resample the training data with SMOTEENN
smote_enn = SMOTEENN(random_state=0)
X_smote_enn, y_smote_enn = smote_enn.fit_resample(X_train_scaled, y_train)

# View the count of target classes with Counter
Counter(y_smote_enn)

Counter({0: 49791, 1: 49984})

In [159]:
# Train the Logistic Regression model using the resampled data
combination_lr_model = LogisticRegression(solver='lbfgs', random_state=1)
combination_lr_model.fit(X_smote_enn, y_smote_enn)

LogisticRegression(random_state=1)

In [161]:
# Calculate the balanced accuracy score
y_combination_pred = combination_lr_model.predict(X_test_scaled)
balanced_accuracy_score(y_test, y_combination_pred)

0.9944146206622002

In [164]:
# Display the confusion matrix
cm_combination = confusion_matrix(y_test, y_combination_pred)
cm_combination_df = pd.DataFrame(cm_combination, index=["Actual 0", "Actual 1"], columns=["Predicted 0","Predicted 1"])
display(cm_combination_df)

,Predicted 0,Predicted 1
Actual 0,839,4
Actual 1,159,24585


In [165]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_combination_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.84      1.00      0.99      0.91      0.99      0.99       843
          1       1.00      0.99      1.00      1.00      0.99      0.99     24744

avg / total       0.99      0.99      1.00      0.99      0.99      0.99     25587



# Final Questions

1. Which model had the best balanced accuracy score?

   YOUR ANSWER HERE.

2. Which model had the best recall score?

    YOUR ANSWER HERE.

3. Which model had the best geometric mean score?

    YOUR ANSWER HERE.


# My Thoughts
1. Out of the several sampling models, the Combination (oversample & undersample) model had the best accuracy score with 99.44%. This was slightly better than the SMOTE oversample model
2. The highest recall score resulted in a tie between the combination, random over sampler, and SMOTE models. They all performed very similarly.
3. They all performed pretty evenly when it comes to the geometric score.

## In the end, I would go with the combination sampler to avoid oversampling.